### Restart and Run All

In [1]:
import pandas as pd
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine('sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3')
conlt = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)
pd.set_option('display.max_rows',None)

year = 2023
quarter = 2

today = date.today()
today

datetime.date(2023, 8, 19)

In [2]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s)
OR (year = %s-1 AND quarter >= %s+1) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
dfc = pd.read_sql(sql, conlt)
dfc['Counter'] = 1
dfc_grp = dfc.groupby(['name'], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp['Counter'] == 4]
dfc_grp

,name,year,quarter,q_amt,Counter
0,ACE,8090,10,1166351,4
1,ADVANC,8090,10,27332445,4
3,AH,8090,10,1983193,4
4,AIE,8090,10,-186609,4
5,AIMIRT,8090,10,693288,4
6,AIT,8090,10,473640,4
7,AJ,8090,10,-536546,4
8,AMATA,8090,10,1944361,4
9,ANAN,8090,10,-115177,4
10,AOT,8090,10,-1336443,4


In [3]:
sql = '''
SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = %s AND quarter <= %s-1) 
OR (year = %s-1 AND quarter >= %s) 
ORDER BY year DESC, quarter DESC'''
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt 
FROM epss 
WHERE (year = 2023 AND quarter <= 2-1) 
OR (year = 2023-1 AND quarter >= 2) 
ORDER BY year DESC, quarter DESC


In [4]:
dfp = pd.read_sql(sql, conlt)
dfp['Counter'] = 1
dfp_grp = dfp.groupby(['name'], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp['Counter'] == 4]
dfp_grp.shape

(231, 5)

In [5]:
dfm = pd.merge(dfc_grp, dfp_grp, on='name', suffixes=(['_c','_p']), how='inner')
dfm['inc_profit'] = dfm['q_amt_c']-dfm['q_amt_p']
dfm['Pct'] = round(dfm['inc_profit']/abs(dfm['q_amt_p'])*100,2)
dfm['year'] = year
dfm['quarter'] = 'Q'+str(quarter)
df_pct = dfm[['name','year','quarter','q_amt_c','q_amt_p','inc_profit','Pct']]
df_pct

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,ACE,2023,Q2,1166351,1234293,-67942,-5.50
1,ADVANC,2023,Q2,27332445,26457375,875070,3.31
2,AH,2023,Q2,1983193,1983003,190,0.01
3,AIE,2023,Q2,-186609,-151353,-35256,-23.29
4,AIMIRT,2023,Q2,693288,682646,10642,1.56
5,AIT,2023,Q2,473640,472255,1385,0.29
6,AJ,2023,Q2,-536546,-306699,-229847,-74.94
7,AMATA,2023,Q2,1944361,2279873,-335512,-14.72
8,ANAN,2023,Q2,-115177,-74085,-41092,-55.47
9,AOT,2023,Q2,-1336443,-6473437,5136994,79.35


In [6]:
sql = """
DELETE FROM qt_profits 
WHERE year = %s AND quarter = 'Q%s'"""
sql = sql % (year,quarter)
print(sql)


DELETE FROM qt_profits 
WHERE year = 2023 AND quarter = 'Q2'


In [7]:
rp = conlt.execute(sql)
rp.rowcount

181

In [8]:
sql = 'SELECT name, id FROM tickers'
tickers = pd.read_sql(sql, conlt)
tickers.shape

(399, 2)

In [9]:
df_ins = pd.merge(df_pct, tickers, on='name', how='inner')
df_ins.dtypes

name           object
year            int64
quarter        object
q_amt_c         int64
q_amt_p         int64
inc_profit      int64
Pct           float64
id              int64
dtype: object

In [10]:
rcds = df_ins.values.tolist()
len(rcds)

211

In [11]:
for rcd in rcds:
    print(rcd)

['ACE', 2023, 'Q2', 1166351, 1234293, -67942, -5.5, 698]
['ADVANC', 2023, 'Q2', 27332445, 26457375, 875070, 3.31, 6]
['AH', 2023, 'Q2', 1983193, 1983003, 190, 0.01, 9]
['AIE', 2023, 'Q2', -186609, -151353, -35256, -23.29, 720]
['AIMIRT', 2023, 'Q2', 693288, 682646, 10642, 1.56, 669]
['AIT', 2023, 'Q2', 473640, 472255, 1385, 0.29, 11]
['AJ', 2023, 'Q2', -536546, -306699, -229847, -74.94, 12]
['AMATA', 2023, 'Q2', 1944361, 2279873, -335512, -14.72, 21]
['ANAN', 2023, 'Q2', -115177, -74085, -41092, -55.47, 23]
['AOT', 2023, 'Q2', -1336443, -6473437, 5136994, 79.35, 24]
['AP', 2023, 'Q2', 5596465, 5625945, -29480, -0.52, 25]
['ASIAN', 2023, 'Q2', 532125, 780969, -248844, -31.86, 36]
['ASK', 2023, 'Q2', 1541463, 1563900, -22437, -1.43, 38]
['ASP', 2023, 'Q2', 536435, 460634, 75801, 16.46, 40]
['ASW', 2023, 'Q2', 1059831, 1175597, -115766, -9.85, 728]
['AWC', 2023, 'Q2', 4976058, 4630796, 345262, 7.46, 699]
['BA', 2023, 'Q2', 1302119, -215102, 1517221, 705.35, 45]
['BAM', 2023, 'Q2', 2274028

In [12]:
sql = """
INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) \
 VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO qt_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id)  VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [13]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [14]:
cols = 'name year quarter q_amt_c q_amt_p inc_profit Pct'.split()

In [15]:
criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
16,BA,2023,Q2,1302119,-215102,1517221,705.35
61,EGCO,2023,Q2,2826239,589487,2236752,379.44
106,MAJOR,2023,Q2,699407,298236,401171,134.51
139,ROJNA,2023,Q2,954903,509514,445389,87.41
182,TIPH,2023,Q2,1810466,1173087,637379,54.33


In [16]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
61,EGCO,2023,Q2,2826239,589487,2236752,379.44
139,ROJNA,2023,Q2,954903,509514,445389,87.41
182,TIPH,2023,Q2,1810466,1173087,637379,54.33
208,WHAUP,2023,Q2,909289,633231,276058,43.60
111,MINT,2023,Q2,8797837,7104207,1693630,23.84


In [17]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
16,BA,2023,Q2,1302119,-215102,1517221,705.35
61,EGCO,2023,Q2,2826239,589487,2236752,379.44
106,MAJOR,2023,Q2,699407,298236,401171,134.51
30,BGRIM,2023,Q2,3082,-868459,871541,100.35
139,ROJNA,2023,Q2,954903,509514,445389,87.41


In [18]:
df_ins_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[df_ins_criteria,cols].sort_values(['Pct'],ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
61,EGCO,2023,Q2,2826239,589487,2236752,379.44
139,ROJNA,2023,Q2,954903,509514,445389,87.41
182,TIPH,2023,Q2,1810466,1173087,637379,54.33
208,WHAUP,2023,Q2,909289,633231,276058,43.60
111,MINT,2023,Q2,8797837,7104207,1693630,23.84
